In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, get_scheduler
from bitsandbytes.optim import Adam8bit,PagedAdam32bit
from peft import LoraConfig, get_peft_model
from peft import prepare_model_for_kbit_training
import torch
from IPython.display import  clear_output
import time
import gc
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False,
    )


model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    quantization_config=bnb_config,
    use_safetensors=True,
    device_map=device,
)

print(model.get_memory_footprint()/(1024*1024))

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

def flush():
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2095.841064453125


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("OdiaGenAI/hardcode_odia_qa_105")

In [3]:
dataset['train'][0]

{'input': '',
 'english_input': '',
 'english_instruction': 'Who are you?',
 'instruction': 'ଆପଣ କିଏ?',
 'output': 'ମୁଁ ଅଲିଭ୍ ଏକ ଚାଟ୍ବଟ୍ ଆସିଷ୍ଟାଣ୍ଟ, ଯାହାକି ଓଡିଆ-ଜେନ-ଏ.ଆଇ. ଗବେଷକମାନଙ୍କ ଦ୍ୱାରା ପ୍ରଶିକ୍ଷିତ ଏକ ଭାଷା ମଡେଲ।',
 'english_output': 'I am Olive a chatbot assistant, a language model trained by researchers from OdiaGenAI.'}

In [4]:
# Custom PyTorch Dataset
class LlamaDataset(Dataset):
    def __init__(self, dataset):
        self.data = dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        question=sample['instruction']
        answer = sample['output']
        chat_template = f'''<|begin_of_text|> <|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{answer}।<|eot_id|>'''
        inputs = tokenizer(chat_template,return_tensors='pt',truncation=True,padding="max_length",max_length=380)
        labels = torch.where(inputs.input_ids==tokenizer.pad_token_id,-100,inputs.input_ids)
        
        input_ids = inputs.input_ids[:,:-1].squeeze()
        labels = labels[:, 1:].squeeze()
        
        return input_ids,labels

In [5]:
train_dataset = LlamaDataset(dataset['train'])
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [6]:
for param in model.named_parameters():
    print(param[0],param[1].dtype)

model.embed_tokens.weight torch.float16
model.layers.0.self_attn.q_proj.weight torch.uint8
model.layers.0.self_attn.k_proj.weight torch.uint8
model.layers.0.self_attn.v_proj.weight torch.uint8
model.layers.0.self_attn.o_proj.weight torch.uint8
model.layers.0.mlp.gate_proj.weight torch.uint8
model.layers.0.mlp.up_proj.weight torch.uint8
model.layers.0.mlp.down_proj.weight torch.uint8
model.layers.0.input_layernorm.weight torch.float16
model.layers.0.post_attention_layernorm.weight torch.float16
model.layers.1.self_attn.q_proj.weight torch.uint8
model.layers.1.self_attn.k_proj.weight torch.uint8
model.layers.1.self_attn.v_proj.weight torch.uint8
model.layers.1.self_attn.o_proj.weight torch.uint8
model.layers.1.mlp.gate_proj.weight torch.uint8
model.layers.1.mlp.up_proj.weight torch.uint8
model.layers.1.mlp.down_proj.weight torch.uint8
model.layers.1.input_layernorm.weight torch.float16
model.layers.1.post_attention_layernorm.weight torch.float16
model.layers.2.self_attn.q_proj.weight tor

In [7]:
model = prepare_model_for_kbit_training(model)

In [8]:
for param in model.named_parameters():
    print(param[0],param[1].dtype)

model.embed_tokens.weight torch.float32
model.layers.0.self_attn.q_proj.weight torch.uint8
model.layers.0.self_attn.k_proj.weight torch.uint8
model.layers.0.self_attn.v_proj.weight torch.uint8
model.layers.0.self_attn.o_proj.weight torch.uint8
model.layers.0.mlp.gate_proj.weight torch.uint8
model.layers.0.mlp.up_proj.weight torch.uint8
model.layers.0.mlp.down_proj.weight torch.uint8
model.layers.0.input_layernorm.weight torch.float32
model.layers.0.post_attention_layernorm.weight torch.float32
model.layers.1.self_attn.q_proj.weight torch.uint8
model.layers.1.self_attn.k_proj.weight torch.uint8
model.layers.1.self_attn.v_proj.weight torch.uint8
model.layers.1.self_attn.o_proj.weight torch.uint8
model.layers.1.mlp.gate_proj.weight torch.uint8
model.layers.1.mlp.up_proj.weight torch.uint8
model.layers.1.mlp.down_proj.weight torch.uint8
model.layers.1.input_layernorm.weight torch.float32
model.layers.1.post_attention_layernorm.weight torch.float32
model.layers.2.self_attn.q_proj.weight tor

In [9]:
config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1,
    bias="none",
    inference_mode=False,
    use_rslora=True,
    init_lora_weights="gaussian",
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 36,700,160 || all params: 3,249,449,984 || trainable%: 1.1294


In [10]:
for param in model.named_parameters():
    print(param[0],param[1].requires_grad)

base_model.model.model.embed_tokens.weight False
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight False
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight True
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight True
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight False
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight True
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight True
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight False
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight True
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight True
base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight False
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight True
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight True
base_model.model.model.layers.0.mlp.gate_proj.weigh

In [11]:
# question = '''ଓଡ଼ିଶାରେ ପର୍ଯ୍ୟଟନକୁ ପ୍ରୋତ୍ସାହନ ଦେବା ପାଇଁ ସରକାରୀ ଏବଂ ଘରୋଇ କ୍ଷେତ୍ର କିପରି ମିଳିମିଶି କାର୍ଯ୍ୟ କରିପାରିବେ?'''
# answer = '''ଯେକୌଣସି ରାଜ୍ୟରେ ପର୍ଯ୍ୟଟନକୁ ପ୍ରୋତ୍ସାହନ ଦେବା ପାଇଁ ସରକାରୀ ଏବଂ ଘରୋଇ କ୍ଷେତ୍ରର ମିଳିତ ପ୍ରୟାସର ଆବଶ୍ୟକତା ରହିଛି। ଓଡ଼ିଶାର ସରକାରୀ ଏବଂ ଘରୋଇ କ୍ଷେତ୍ର ରାଜ୍ୟରେ ପର୍ଯ୍ୟଟନକୁ ପ୍ରୋତ୍ସାହନ ଦେବା ଲାଗି ଏକ ବିସ୍ତୃତ ରଣନୀତି ବିକଶିତ ଏବଂ କାର୍ଯ୍ୟକାରୀ କରିବା ଲାଗି ମିଳିତ ଭାବେ କାର୍ଯ୍ୟ କରିପାରିବେ।
# ସରକାର ଘରୋଇ କ୍ଷେତ୍ର ସହିତ ମିଶି କାମ କରିବାର ଗୋଟିଏ ଉପାୟ ହେଲା ଘରୋଇ ନିବେଶ ପାଇଁ ଅନୁକୂଳ ବାତାବରଣ ସୃଷ୍ଟି କରିବା, ଏଥିରେ ଘରୋଇ କ୍ଷେତ୍ରର ଭାଗିଦାରୀକୁ ପ୍ରୋତ୍ସାହିତ କରିବା ପାଇଁ ଟିକସ ଏବଂ ନିୟାମକ ପ୍ରତିବନ୍ଧକକୁ ହ୍ରାସ କରିବା, ଏହା ବ୍ୟତୀତ ସରକାର ଟିକସ ରିହାତି, ସବସିଡି ଏବଂ ପର୍ଯ୍ୟଟନ ବିକାଶ ପ୍ରକଳ୍ପ ପାଇଁ ଜମି ଆଦି ପ୍ରୋତ୍ସାହନ ମଧ୍ୟ ପ୍ରଦାନ କରିପାରିବେ।
# ଘରୋଇ କ୍ଷେତ୍ର ସହ ମିଶି ସରକାର ଘରୋଇ କ୍ଷେତ୍ର ସହ ମିଶି ନୂତନ ପର୍ଯ୍ୟଟନ ଉତ୍ପାଦ ପ୍ରସ୍ତୁତ କରିପାରିବେ ଯାହା ଉଭୟ ଘରୋଇ ଏବଂ ଅନ୍ତର୍ଜାତୀୟ ପର୍ଯ୍ୟଟକଙ୍କ ଆବଶ୍ୟକତା ପୂରଣ କରିପାରିବ।
# ସରକାର ମଧ୍ୟ ଓଡ଼ିଶାରେ ପର୍ଯ୍ୟଟନକୁ ପ୍ରୋତ୍ସାହନ ଦେବା ଏବଂ ଘରୋଇ କ୍ଷେତ୍ରର ଭାଗିଦାରୀକୁ ପ୍ରୋତ୍ସାହିତ କରିବା ପାଇଁ ପ୍ରଯୁକ୍ତିର ଉପଯୋଗ କରିପାରିବେ। ଉଦାହରଣ ସ୍ୱରୂପ, ସରକାର ପର୍ଯ୍ୟଟନ ସ୍ଥଳକୁ ପ୍ରୋତ୍ସାହିତ କରିବା ଏବଂ ସମ୍ଭାବ୍ୟ ପର୍ଯ୍ୟଟକମାନଙ୍କ ସହିତ ଯୋଡ଼ିବା ଲାଗି ସୋସିଆଲ ମିଡିଆ ପ୍ଲାଟଫର୍ମର ଉପଯୋଗ କରିପାରିବେ। ଓଡ଼ିଶାରେ ପର୍ଯ୍ୟଟନ ଆକର୍ଷଣ ଏବଂ ଅନୁଭବ ପ୍ରଦର୍ଶିତ କରିବା ଲାଗି ଏକ ଅନଲାଇନ ପ୍ଲାଟଫର୍ମ ପ୍ରତିଷ୍ଠା କରିବା ଦ୍ୱାରା ଅଧିକ ପର୍ଯ୍ୟଟକଙ୍କୁ ଆକର୍ଷିତ କରିବାରେ ସହାୟତା ମିଳିପାରିବ।
# ପର୍ଯ୍ୟଟନକୁ ପ୍ରୋତ୍ସାହନ ଦେବା ଲାଗି ସରକାରୀ ଏବଂ ଘରୋଇ କ୍ଷେତ୍ରକୁ ମିଳିତ ଭାବେ କାର୍ଯ୍ୟ କରିବାକୁ ପଡିବ ଯେପରିକି ପର୍ଯ୍ୟଟନ ଗତିବିଧି ଦ୍ୱାରା ପର୍ଯ୍ୟାବରଣର କ୍ଷୟ କିମ୍ବା ସ୍ଥାନୀୟ ସମ୍ପ୍ରଦାୟର କ୍ଷତି ନ ହେଉ।
# ଶେଷରେ, ସରକାରୀ ଏବଂ ଘରୋଇ କ୍ଷେତ୍ରକୁ ଓଡ଼ିଶାରେ ପର୍ଯ୍ୟଟନକୁ ପ୍ରୋତ୍ସାହନ ଦେବା ଲାଗି ଏକ ଅନୁକୂଳ ପରିବେଶ ସୃଷ୍ଟି କରିବା ଆବଶ୍ୟକ ଏବଂ ଏହା ସୁନିଶ୍ଚିତ କରିବା ଉଚିତ ଯେ ବିକାଶ ସ୍ଥାୟୀ ହେବ। ” ମିଳିତ ଭାବେ କାର୍ଯ୍ୟ କରି ସେମାନେ ପର୍ଯ୍ୟଟନ ରଣନୀତିକୁ ବିକଶିତ ଏବଂ କାର୍ଯ୍ୟକାରୀ କରିପାରିବେ ଯାହା କେବଳ ପର୍ଯ୍ୟଟନ ଉଦ୍ୟୋଗ ନୁହେଁ ବରଂ ସ୍ଥାନୀୟ ଗୋଷ୍ଠୀ ଏବଂ ପରିବେଶକୁ ମଧ୍ୟ ଲାଭାନ୍ୱିତ କରିବ।'''

# tokenized_text = tokenizer(answer).input_ids
# print(len(tokenized_text))
# for idx in range(len(tokenized_text)):
#     clear_output(wait=True)
#     print(tokenizer.decode(tokenized_text[0:idx]))
#     time.sleep(0.1)

# Finetune the LLAMA model on a single text

In [12]:
for input_ids,labels in train_dataloader:
    break

tokenizer.batch_decode(input_ids)

['<|begin_of_text|><|begin_of_text|> <|start_header_id|>user<|end_header_id|>\n\nନମସ୍କାର।<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nମୁଁ ଅଲିଭ୍ ଏକ ଚାଟ୍ବଟ୍ ଆସିଷ୍ଟାଣ୍ଟ ନାମକ ଏକ ଭାଷା ମଡେଲ୍ ଏବଂ ଓଡିଆ-ଜେନ-ଏ.ଆଇ. ଗବେଷକମାନଙ୍କ ଦ୍ୱାରା ପ୍ରଶିକ୍ଷିତ ହୋଇଛି।।<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot

In [13]:
def generate_eval():
    sample=dataset['train'][0]
    question=sample['instruction']
    answer = sample['output']
    chat_template = f'''<|begin_of_text|> <|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|>'''
    inputs = tokenizer(chat_template , return_tensors="pt").to(device)
    # print(prompt)

    model.eval()

    with torch.no_grad():
        output = model.generate(
            **inputs,
            do_sample=True,
            max_new_tokens=512,
            repetition_penalty=1.3,
            temperature=0.7,         # Optional: smooth randomness
            top_k=50,                # Optional: top-k sampling
            top_p=0.9                # Optional: nucleus sampling
        )

    processed_text = tokenizer.decode(output[0], skip_special_tokens=False)
    
    model.train()

    return processed_text

In [14]:
pred = generate_eval()
print(pred)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|begin_of_text|> <|start_header_id|>user<|end_header_id|>

ଆପଣ କିଏ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"ମേ'ୟ ତാ. ନુ' ଯଥ ଚ‌‍​ਡ́ੜ 'ଁ ଓ ସ‌‍​ਲ; ਡ്ര̀ ଶ‌‍​ਖ ଅ ଗ‌‍ ​ਹ̌ ਸ‌‍ ​​'🙃".<|eot_id|>


In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
gradient_accumulation_steps = 4
max_steps=500
# Define optimizer
optimizer = PagedAdam32bit(model.parameters(), lr=1e-4)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_steps,
)
# Training loop
model.train()

global_step= 0

while global_step< 500:
    for step,(input_ids, labels) in enumerate(train_dataloader):
        input_ids, labels= input_ids.to('cuda'),labels.to('cuda')
        model.config.use_cache = False
        model.train()
        
        
        # Forward pass
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss = loss / gradient_accumulation_steps  # Normalize loss
        loss.backward()
        
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
        
        global_step += 1
        if global_step >= max_steps:
            break
        
        if global_step % 20 == 0:
            pred = generate_eval()
            print('*'*20,step+1,'*'*20)
            print("Predictions:", pred)
            print('*'*20,'end','*'*20)
            
        
        print(f"Epoch {global_step + 1}/{max_steps}, Loss: {loss.item():.4f}")
        
flush()

Epoch 2/500, Loss: 3.4890


In [16]:
save_path = "/home/nas/buffer/mohan.dash/llama_3_finetuned/model_checkpoint.pt"

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'lr_scheduler_state_dict': lr_scheduler.state_dict(),
    'global_step': global_step
}, save_path)

print(f"Checkpoint saved to {save_path}")

Checkpoint saved to /home/nas/buffer/mohan.dash/llama_3_finetuned/model_checkpoint.pt
